<a href="https://colab.research.google.com/github/Pinalli/T2_IA/blob/master/T2_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from nltk import word_tokenize
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from nltk.corpus import sentiwordnet as swn
import pandas as pd
df = pd.read_csv('debate_3000.csv', encoding='utf-8')
#breakdown = swn.senti_synset(df)
#breakdown = swn.senti_synset('breakdown.n.03')
print(df)

                                                   text
0     EdegarPretto O meu candidato vice-governador é...
1     Periga o Argenta chamar o Simon pra responder ...
2      WinstonLing Segue o link para assistirem o de...
3        EdegarPretto Eu quero dizer aos gaúchos e g...
4      wmfigueiredo  eve onvf  bandrs  EduardoLeite_...
...                                                 ...
2995     josenery53  SigaPepeVargas  DilmaResiste  E...
2996                           onyxlorenzoni Realmente 
2997     bandrs Band mantém tradição e realiza o pri...
2998     EdegarPretto O debate começou e você pode a...
2999   EdegarPretto tem 51 anos e nasceu em Miraguaí...

[3000 rows x 1 columns]


In [ ]:
df.head()

,retweet_count;text
0,45;RT EdegarPretto O meu candidato vice-gover...
1,0;Periga o Argenta chamar o Simon pra responde...
2,14;RT WinstonLing Segue o link para assistire...
3,88;RT EdegarPretto Eu quero dizer aos gaúchos...
4,0; wmfigueiredo evertonvf bandrs EduardoLei...


## Pré - Processamento dos Dados

*  Remove linhas duplicadas na base de dados
    + Problemas na coleta de dados
* Remoce StopWords
* Stemming ou Lemmatization
* Remove caracteres

In [11]:
df.drop_duplicates( inplace=True)
print(df)

                                                   text
0     EdegarPretto O meu candidato vice-governador é...
1     Periga o Argenta chamar o Simon pra responder ...
2      WinstonLing Segue o link para assistirem o de...
3        EdegarPretto Eu quero dizer aos gaúchos e g...
4      wmfigueiredo  eve onvf  bandrs  EduardoLeite_...
...                                                 ...
2988     ptsul  economia solidária está incluída nos...
2989     ptsul Os jovens da roça não querem ser agri...
2991   vitoriafamer Uma ótima opo unidade para o pes...
2995     josenery53  SigaPepeVargas  DilmaResiste  E...
2996                           onyxlorenzoni Realmente 

[1818 rows x 1 columns]


In [ ]:
df.count()

retweet_count;text    661
dtype: int64

## Sepando tweets e suas classes

In [14]:
tweets = df['text']
print(tweets)

0       EdegarPretto O meu candidato vice-governador é...
1       Periga o Argenta chamar o Simon pra responder ...
2        WinstonLing Segue o link para assistirem o de...
3          EdegarPretto Eu quero dizer aos gaúchos e g...
4        wmfigueiredo  eve onvf  bandrs  EduardoLeite_...
                              ...                        
2988       ptsul  economia solidária está incluída nos...
2989       ptsul Os jovens da roça não querem ser agri...
2991     vitoriafamer Uma ótima opo unidade para o pes...
2995       josenery53  SigaPepeVargas  DilmaResiste  E...
2996                             onyxlorenzoni Realmente 
Name: text, Length: 1818, dtype: object


## Instalação bibliotecas


In [16]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Funçoes Pre-processamento



In [18]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))#carrega a lista de stopwords em portugues
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [19]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split(): #reduz o sufixo => frequentemente = frequente
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

  

In [20]:
def Limpa_dados(instancia):  #remove links, (. , ; ())
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.', '').replace(';','').replace('-', '').replace(',', '')
    return (instancia)


In [21]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def Lemmatization(instancia): #mais próximo da realidade da palavra
    palavras = []
    for w in instancia.split():
      palavras.append(wordnet_lemmatizer.lemmatize(w))
    return (" ".join(palavras))



## Defs funcionando

In [22]:
RemoveStopWords("EdegarPretto Eu quero dizer aos gaúchos e gaúchas que se eu governador vou anular todos os atos de privatização da nossa Corsan…")

'EdegarPretto Eu quero dizer gaúchos gaúchas governador vou anular todos atos privatização Corsan…'

In [23]:
Stemming("EdegarPretto Eu quero dizer aos gaúchos e gaúchas que se eu governador vou anular todos os atos de privatização da nossa Corsan…")

'edegarprett eu quer diz ao gaúch e gaúch que se eu govern vou anul tod os ato de priva da noss corsan…'

In [27]:
Stemming("EdegarPretto Eu quero dizer aos gaúchos e gaúchas que se eu governador vou anular todos os atos de privatização da nossa Co")

'edegarprett eu quer diz ao gaúch e gaúch que se eu govern vou anul tod os ato de priva da noss co'

In [30]:
Limpa_dados("EdegarPretto Eu quero dizer aos gaúchos e gaúchas, que se eu governador vou anular todos os atos de privatização da nossa Corsan…")

'edegarpretto eu quero dizer aos gaúchos e gaúchas, que se eu governador vou anular todos os atos de privatização da nossa corsan…'